In [26]:
import pandas as pd

In [15]:
### cargar data intermedia que contiene todos los campos 4 bases anteriores
df_intermediate = pd.read_pickle('../data/df_intermediate_v3.pkl')
df_intermediate.head()

,Cliente,Marca,Cupo,CodigoDC,Is_Executed,Fecha_Desde,Fecha_Hasta,Duracion_Promocion,Cliente_Sales,FechaAltaCliente,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr,Hl,Dcto,Año,Mes,ClaseEnvase,SegmentoPrecio,ListMarca,ListCupo
Cliente_Marca_Cupo,,,,,,,,,,,,,,,,,,,,,,,,,
3213299,3213,29,9,"[297601, 354404, 619037, 619621, 542054]","[0, 0, 0, 0, 0]","[2018-08-06 00:00:00, 2018-09-03 00:00:00, 201...","[2018-08-30 00:00:00, 2018-09-27 00:00:00, 201...","[24, 24, 5, 3, 3]",2,2010-03-29,3,8,15,2,3,1,"[29082.62, 16184.87, 76800.02, 28110.53, 9076....","[0.09, 0.09, 0.25, 0.09, 0.02, 0.03, 0.03, 0.0...","[0.0, 0.0, 0.0, -1333.83, 0.0, 0.0, 0.0, -621....","[2018, 2018, 2018, 2018, 2018, 2018, 2018, 201...","[2, 3, 3, 3, 4, 4, 5, 5, 5, 6, 6, 6, 6, 6, 7, ...","[2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, ...","[1, 2, 1, 3, 1, 3, 1, 3, 3, 1, 1, 2, 1, 1, 1, ...","[32, 44, 32, 23, 28, 23, 2, 23, 23, 2, 40, 37,...","[17, 29, 17, 16, 25, 26, 26, 26, 16, 26, 22, 3..."
3795299,3795,29,9,"[297601, 354404, 542055, 619684, 644816, 61843...","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[2018-08-06 00:00:00, 2018-09-03 00:00:00, 201...","[2018-08-30 00:00:00, 2018-09-27 00:00:00, 201...","[24, 24, 3, 3, 7, 5, 17, 9, 9]",4,2010-04-26,3,10,6,2,3,1,"[84189.6, 115610.44, 106536.01, 19009.21, 9504...","[0.26, 0.36, 0.13, 0.05, 0.03, 0.05, 0.03, 0.6...","[0.0, 0.0, -9264.0, 0.0, 0.0, -1079.72, -754.7...","[2018, 2018, 2018, 2018, 2018, 2018, 2018, 201...","[1, 1, 2, 3, 3, 4, 4, 5, 6, 6, 7, 9, 12, 1, 2,...","[1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, ...","[1, 1, 4, 3, 3, 3, 3, 1, 4, 1, 1, 1, 3, 4, 4, ...","[40, 40, 30, 23, 26, 23, 23, 40, 17, 2, 6, 2, ...","[22, 16, 18, 26, 26, 26, 16, 22, 12, 16, 16, 1..."
11816299,11816,29,9,"[297601, 354404, 646169, 659921, 660481, 542055]","[1, 0, 0, 0, 0, 0]","[2018-08-06 00:00:00, 2018-09-03 00:00:00, 201...","[2018-08-30 00:00:00, 2018-09-27 00:00:00, 201...","[24, 24, 7, 9, 9, 3]",1,2017-10-01,3,7,12,2,3,1,"[91605.99, 48655.49, 64183.96, 39308.15, 49778...","[0.33, 0.15, 0.21, 0.12, 0.07, 0.24, 0.09, 1.6...","[0.0, 0.0, 0.0, 0.0, -4328.55, 0.0, 0.0, -2276...","[2018, 2018, 2018, 2018, 2018, 2018, 2018, 201...","[1, 1, 2, 2, 3, 4, 5, 5, 6, 6, 6, 7, 7, 7, 7, ...","[2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, ...","[1, 3, 1, 3, 4, 3, 1, 1, 4, 1, 1, 1, 1, 1, 3, ...","[2, 46, 6, 26, 31, 23, 32, 40, 17, 6, 40, 4, 7...","[16, 16, 16, 16, 8, 16, 17, 16, 12, 16, 16, 16..."
84444016,8444,40,16,"[298856, 355104, 357820, 360816, 354320, 40554...","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[2018-08-12 00:00:00, 2018-09-05 00:00:00, 201...","[2018-08-19 00:00:00, 2018-09-06 00:00:00, 201...","[7, 1, 4, 13, 1, 6, 9, 0, 9, 5, 8, 3, 3, 5, 13...",15,2015-09-02,5,27,18,2,2,1,"[31322.21, 34399.16, 27663.75, 87380.26, 18442...","[0.09, 0.19, 0.11, 0.41, 0.07, 0.5, 0.06, 0.05...","[-316.39, -28121.85, -723.69, -48000.0, -482.4...","[2018, 2018, 2018, 2018, 2018, 2018, 2018, 201...","[1, 1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 4, 4, 5, 5, ...","[1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, ...","[2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, ...","[44, 44, 6, 40, 6, 40, 44, 28, 37, 7, 6, 28, 4...","[32, 33, 19, 16, 19, 16, 16, 30, 16, 25, 19, 1..."
84884016,8488,40,16,"[309916, 355700, 354347, 354783, 374390, 35643...","[1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, ...","[2018-08-12 00:00:00, 2018-09-06 00:00:00, 201...","[2018-08-19 00:00:00, 2018-09-06 00:00:00, 201...","[7, 0, 13, 1, 5, 4, 22, 6, 0, 11, 4, 9, 5, 0, ...",12,2015-09-16,3,10,16,2,2,0,"[84809.55, 39496.79, 20699.98, 68551.66, 16521...","[0.1, 0.13, 0.09, 0.2, 0.06, 0.03, 0.06, 0.09,...","[-7459.2, 0.0, -3652.94, 0.0, 0.0, 0.0, -714.2...","[2018, 2018, 2018, 2018, 2018, 2018, 2018, 201...","[1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 5, 5, 5, ...","[1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, ...","[4, 1, 2, 1, 1, 2, 3, 1, 1, 3, 3, 1, 2, 1, 1, ...","[30, 32, 37, 40, 28, 37, 24, 28, 40, 23, 26, 2...","[18, 17, 16, 22, 16, 16, 16, 16, 19, 16, 16, 1..."


In [16]:
import statistics as stat
def calculate_mode(x):
  try:
    moda=stat.mode(x)
  except:
    moda=0
  return moda

In [ ]:
import numpy as np
df_intermediate['NroPromociones'] = df_intermediate['CodigoDC'].apply(lambda x: len(x))
df_intermediate['NroPromocionesConsumidas'] = df_intermediate['Is_Executed'].apply(lambda x: sum(x))
df_intermediate['NroPromocionesRatio'] = df_intermediate['NroPromocionesConsumidas']/df_intermediate['NroPromociones'] 
df_intermediate['Target'] = df_intermediate['Is_Executed'].apply(lambda x: 1 if sum(x)>0 else 0)
df_intermediate['Duracion_Promocion_Max'] = df_intermediate['Duracion_Promocion'].apply(lambda x: max(x))
df_intermediate['Duracion_Promocion_Mean'] = df_intermediate['Duracion_Promocion'].apply(lambda x: np.mean(x))
df_intermediate['Nr_Total'] = df_intermediate['Nr'].apply(lambda x: sum(x))
df_intermediate['Nr_Promedio'] = df_intermediate['Nr'].apply(lambda x: np.mean(x))
df_intermediate['Nr_Std'] = df_intermediate['Nr'].apply(lambda x: np.std(x))
df_intermediate['Nro_Compra'] = df_intermediate['Nr'].apply(lambda x: len(x))
df_intermediate['Hl_Total'] = df_intermediate['Hl'].apply(lambda x: sum(x))
df_intermediate['Hl_Promedio'] = df_intermediate['Hl'].apply(lambda x: np.mean(x))
df_intermediate['Dcto_Total'] = df_intermediate['Dcto'].apply(lambda x: sum(x))
df_intermediate['Dcto_Promedio'] = df_intermediate['Dcto'].apply(lambda x: np.mean(x))
df_intermediate['Mes_Moda'] = df_intermediate['Mes'].apply(lambda x:calculate_mode(x))
df_intermediate['ClaseEnvase_Moda'] = df_intermediate['ClaseEnvase'].apply(lambda x:calculate_mode(x))
df_intermediate['SegmentoPrecio_Moda'] = df_intermediate['SegmentoPrecio'].apply(lambda x:calculate_mode(x))
df_intermediate['Marca_Moda'] = df_intermediate['ListMarca'].apply(lambda x:calculate_mode(x))
df_intermediate['Cupo_Moda'] = df_intermediate['ListCupo'].apply(lambda x:calculate_mode(x))

In [18]:
### obtener cuales son los meses mas frecuentes en la que el cliente compra productos de la empresa de cervecería
### obtener cuales son los meses envases mas frecuentes en la que el cliente compra productos de la empresa de cervecería
### obtenerlos segmentos de precios  mas frecuentes a la que pertenece el cliente

def get_mes(n,arr):
  cont = 0
  for i in arr:
    if n == i:
      cont=cont+1
  return cont
for i in range(1,13,1):
  df_intermediate["mes_"+str(i)] = df_intermediate["Mes"].apply(lambda x: get_mes(i, x))
for i in range(1,3,1):
  df_intermediate["ClaseEnvases_"+str(i)] = df_intermediate["ClaseEnvase"].apply(lambda x: get_mes(i, x))
for i in range(1,5,1):
  df_intermediate["SegmentoPrecio_"+str(i)] = df_intermediate["SegmentoPrecio"].apply(lambda x: get_mes(i, x))

def get_list_marca(x):
  dict_ = {40:0, 39:0, 29:0}
  for i in x:
    if i in [40,39,29]:
      dict_[i] +=1
  return dict_

def get_list_cupo(x):
  dict_ = {16:0, 20:0, 9:0}
  for i in x:
    if i in [16,20,9]:
      dict_[i] +=1
  return dict_
### Obtener la frecuencia de las marcas que mas compra el cliente
aux_list_marca = pd.DataFrame(list(df_intermediate['ListMarca'].apply(lambda x: get_list_marca(x))), index=df_intermediate.index)
aux_list_marca.columns = ['Marca_Freq_40', 'Marca_Freq_39', 'Marca_Freq_29']

### Obtener la frecuencia de los cupos que mas compra el cliente
aux_list_cupo = pd.DataFrame(list(df_intermediate['ListCupo'].apply(lambda x: get_list_cupo(x))), index=df_intermediate.index)
aux_list_cupo.columns = ['Cupo_Freq_16', 'Cupo_Freq_20', 'Cupo_Freq_9']
df_intermediate = pd.concat([df_intermediate,aux_list_marca, aux_list_cupo], axis=1)

In [21]:
### Obtener la binarizacion encodes de las Variables Gerencia y Subcanal, esto debido a que estas presentan mas de 20 valores y no optimo hacer un One_Hot_Encoding
import category_encoders as ce
import warnings as w
w.filterwarnings('ignore')
encoder = ce.BinaryEncoder(cols=['Gerencia','SubCanal'])
X_binenc = encoder.fit_transform(df_intermediate[['Gerencia', 'SubCanal']])
X_binenc.head()

,Gerencia_0,Gerencia_1,Gerencia_2,Gerencia_3,Gerencia_4,Gerencia_5,Gerencia_6,SubCanal_0,SubCanal_1,SubCanal_2,SubCanal_3,SubCanal_4,SubCanal_5
Cliente_Marca_Cupo,,,,,,,,,,,,,
3213299,0,0,0,0,0,0,1,0,0,0,0,0,1
3795299,0,0,0,0,0,1,0,0,0,0,0,1,0
11816299,0,0,0,0,0,1,1,0,0,0,0,1,1
84444016,0,0,0,0,1,0,0,0,0,0,1,0,0
84884016,0,0,0,0,0,1,0,0,0,0,1,0,1


In [22]:
#### analizar la variable FechaAltaCliente para encontrar datos relacionados a la antiguedad a la que se encuentra el cliente respecto a su fecha de alta
import datetime as dt
df_intermediate['Annios_Desde_FechaAltaCliente'] = round(df_intermediate['FechaAltaCliente'].apply(lambda x: (dt.datetime.now() - dt.datetime.strptime(x, '%Y-%m-%d')).days)/365, 0)
df_intermediate['Meses_Desde_FechaAltaCliente'] = round(df_intermediate['FechaAltaCliente'].apply(lambda x: (dt.datetime.now() - dt.datetime.strptime(x, '%Y-%m-%d')).days)/30, 0)
df_intermediate['Dias_Desde_FechaAltaCliente'] = df_intermediate['FechaAltaCliente'].apply(lambda x: (dt.datetime.now() - dt.datetime.strptime(x, '%Y-%m-%d')).days)

In [23]:
#### Realizar Onet_Hot_Encoding a las variable de Marca, Cupo, Region, Estrato
df_marca_dummies = pd.get_dummies(df_intermediate['Marca'],prefix='Marca')
df_cupo_dummies = pd.get_dummies(df_intermediate['Cupo'],prefix='Cupo')
df_region_dummies = pd.get_dummies(df_intermediate['Region'],prefix='Region')
df_estrato_dummies = pd.get_dummies(df_intermediate['Estrato'],prefix='Estrato')
df_intermediate = pd.concat([df_intermediate, X_binenc,df_marca_dummies, df_cupo_dummies, df_region_dummies, df_estrato_dummies], axis=1)

In [24]:
df_intermediate.to_pickle('../data/df_feature_engineering_v3.pkl')